### Package Installation

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

### Importing Datasets

In [3]:
patient_dict = {}
for filename in os.listdir('.'):
    if filename.endswith('.csv'):
        df_name = os.path.splitext(filename)[0]  
        patient_dict[df_name] = pd.read_csv(filename)
        print(f"Loaded {df_name}: {patient_dict[df_name].shape}")

Loaded SyncCondition: (3, 3)
Loaded SyncSmokingStatus: (10, 3)
Loaded test_SyncPatient: (3348, 5)
Loaded test_SyncPatientForecast: (3348, 2)
Loaded training_SyncAllergy: (2797, 9)
Loaded training_SyncDiagnosis: (94830, 8)
Loaded training_SyncImmunization: (8, 6)
Loaded training_SyncLabObservation: (29014, 13)
Loaded training_SyncLabPanel: (11139, 5)
Loaded training_SyncLabResult: (1952, 8)
Loaded training_SyncMedication: (44520, 8)
Loaded training_SyncPatient: (9871, 6)
Loaded training_SyncPatientCondition: (2836, 4)
Loaded training_SyncPatientSmokingStatus: (4940, 4)
Loaded training_SyncPatient_orig: (9871, 6)
Loaded training_SyncPrescription: (78863, 9)
Loaded training_SyncTranscript: (131031, 13)
Loaded training_SyncTranscriptAllergy: (801, 3)
Loaded training_SyncTranscriptDiagnosis: (267938, 3)
Loaded training_SyncTranscriptMedication: (97109, 3)


#### Renaming Dataframes

In [4]:
df_train_sp=patient_dict['training_SyncPatient']
df_test_sp=patient_dict['test_SyncPatient']

In [5]:
df_test_sp.columns

Index(['PatientGuid', 'Gender', 'YearOfBirth', 'State', 'PracticeGuid'], dtype='object')

In [6]:
print(df_train_sp.size)
print(df_test_sp.size)

59226
16740


In [7]:
df_sc=patient_dict['SyncCondition']
df_sss=patient_dict['SyncSmokingStatus']
df_train_sa=patient_dict['training_SyncAllergy']
df_train_sd=patient_dict['training_SyncDiagnosis']
df_train_si=patient_dict['training_SyncImmunization']
df_train_slo=patient_dict['training_SyncLabObservation']
df_train_slp=patient_dict['training_SyncLabPanel']
df_train_slr=patient_dict['training_SyncLabResult']
df_train_sm=patient_dict['training_SyncMedication']
df_train_spc=patient_dict['training_SyncPatientCondition']
df_train_spss=patient_dict['training_SyncPatientSmokingStatus']
df_train_spo=patient_dict['training_SyncPatient_orig']
df_train_spr=patient_dict['training_SyncPrescription']
df_train_st=patient_dict['training_SyncTranscript']
df_train_sta=patient_dict['training_SyncTranscriptAllergy']
df_train_std=patient_dict['training_SyncTranscriptDiagnosis']
df_train_stm=patient_dict['training_SyncTranscriptMedication']

In [8]:
for name, df in patient_dict.items():
    print(f"{name}:")
    print(df.head())  
    print("\n")

SyncCondition:
                          ConditionGuid                Code  \
0  D4FF24EA-658D-4500-A71F-821F9776FB27    NOKNOWNDIAGNOSES   
1  4BF4503F-4EA7-4E63-BBA6-B6A4A8C273F8    NOKNOWNALLERGIES   
2  2AE517F6-5029-4BC4-9353-FB8E0CC3A822  NOKNOWNMEDICATIONS   

                   Name  
0    No Known Diagnoses  
1    No Known Allergies  
2  No Known Medications  


SyncSmokingStatus:
                      SmokingStatusGuid  \
0  E86CA3A8-E35B-4BBF-80E2-0375AB4A1460   
1  1F3BFBBF-AB76-481B-B1E0-08A3689A54BC   
2  C12C2DB7-D31A-4514-88C0-42CBD339F764   
3  FA2B7AE4-4D14-4768-A8C7-55B5F0CDF4AF   
4  0815F240-3DD3-43C6-8618-613CA9E41F9F   

                              Description  NISTcode  
0                  Current status unknown         9  
1              Not a current tobacco user         0  
2  0 cigarettes per day (previous smoker)         3  
3            Few (1-3) cigarettes per day         2  
4                  Current status unknown         5  


test_SyncPatient:
    

***Checking Nulls***

In [9]:
for name, df in patient_dict.items():
    null_ct = df.isnull().sum()
    if null_ct.any():  
        print(f"Null value counts for {name}:")
        print(null_ct)
        print("\n")

Null value counts for training_SyncAllergy:
AllergyGuid            0
PatientGuid            0
AllergyType            0
StartYear              0
ReactionName           0
SeverityName           0
MedicationNDCCode    246
MedicationName       272
UserGuid               0
dtype: int64


Null value counts for training_SyncDiagnosis:
DiagnosisGuid               0
PatientGuid                 0
ICD9Code                    0
DiagnosisDescription        0
StartYear                   0
StopYear                90040
Acute                       0
UserGuid                    0
dtype: int64


Null value counts for training_SyncLabObservation:
HL7Identifier          5397
HL7Text                   0
LabObservationGuid        0
LabPanelGuid              0
HL7CodingSystem           0
ObservationValue       4692
Units                     0
ReferenceRange         1163
AbnormalFlags         26797
ResultStatus              0
ObservationYear           0
UserGuid                  0
IsAbnormalValue           0


###### **Imputing Values for Nulls**

In [10]:
for name, df in patient_dict.items():
    df.fillna(0, inplace=True)
    print(f"Null value counts after imputation for {name}:")
    print(df.isnull().sum())
    print("\n")

Null value counts after imputation for SyncCondition:
ConditionGuid    0
Code             0
Name             0
dtype: int64


Null value counts after imputation for SyncSmokingStatus:
SmokingStatusGuid    0
Description          0
NISTcode             0
dtype: int64


Null value counts after imputation for test_SyncPatient:
PatientGuid     0
Gender          0
YearOfBirth     0
State           0
PracticeGuid    0
dtype: int64


Null value counts after imputation for test_SyncPatientForecast:
PatientGuid            0
DMIndicatorForecast    0
dtype: int64


Null value counts after imputation for training_SyncAllergy:
AllergyGuid          0
PatientGuid          0
AllergyType          0
StartYear            0
ReactionName         0
SeverityName         0
MedicationNDCCode    0
MedicationName       0
UserGuid             0
dtype: int64


Null value counts after imputation for training_SyncDiagnosis:
DiagnosisGuid           0
PatientGuid             0
ICD9Code                0
DiagnosisDescrip

***Checking Duplicates***

In [11]:
for name, df in patient_dict.items():
    duplicate_ct = df.duplicated().sum()
    
    if duplicate_ct > 0:
        print(f"Duplicate records found in {name}: {duplicate_ct} duplicates")
    else:
        print(f"No duplicate records in {name}")

No duplicate records in SyncCondition
No duplicate records in SyncSmokingStatus
No duplicate records in test_SyncPatient
No duplicate records in test_SyncPatientForecast
No duplicate records in training_SyncAllergy
No duplicate records in training_SyncDiagnosis
No duplicate records in training_SyncImmunization
No duplicate records in training_SyncLabObservation
No duplicate records in training_SyncLabPanel
No duplicate records in training_SyncLabResult
No duplicate records in training_SyncMedication
Duplicate records found in training_SyncPatient: 3270 duplicates
No duplicate records in training_SyncPatientCondition
No duplicate records in training_SyncPatientSmokingStatus
Duplicate records found in training_SyncPatient_orig: 3270 duplicates
No duplicate records in training_SyncPrescription
No duplicate records in training_SyncTranscript
No duplicate records in training_SyncTranscriptAllergy
No duplicate records in training_SyncTranscriptDiagnosis
No duplicate records in training_SyncT

#### SyncPatient

In [12]:
df_train_sp.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid'],
      dtype='object')

In [13]:
df_train_sp_tot = df_train_sp.groupby("PatientGuid").agg(
    DMIndicator=("DMIndicator", lambda x: x.mode().iloc[0] if not x.mode().empty else None), 
    Gender=("Gender", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    YearOfBirth=("YearOfBirth", lambda x: x.mode().iloc[0] if not x.mode().empty else None), 
    State=("State", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    PracticeGuid=("PracticeGuid", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [14]:
df_train_sp_tot.shape

(6601, 6)

In [15]:
print(df_train_sp_tot['PatientGuid'])

0                                          0
1       00023761-9D8D-445B-874C-2424CC7CF620
2       0005D9BD-0247-4F02-B7EE-7C1B44825FA1
3       00110ABC-DAB9-49E3-A1C8-88BBF8D58109
4       002667F4-B9A3-4DE2-875F-8034DD637865
                        ...                 
6596    FFC9E47E-6971-4210-8370-3FB01BB1B47F
6597    FFCCEB55-7FCA-4CCD-A035-9672674F2353
6598    FFEF293B-6147-47F0-8F0B-76180C124205
6599    FFF2B406-9C3F-4A8B-8B27-7179B7D7FBC6
6600    FFF5C634-1CF7-40D5-BDF1-4FA2D62D9C63
Name: PatientGuid, Length: 6601, dtype: object


In [16]:
df_train_sp_tot = df_train_sp_tot[df_train_sp_tot['PatientGuid'] != 0]

In [17]:
df_train_sp_tot.shape

(6600, 6)

In [18]:
df_train_sp_tot.head()

,PatientGuid,DMIndicator,Gender,YearOfBirth,State,PracticeGuid
1,00023761-9D8D-445B-874C-2424CC7CF620,0.0,M,1956.0,AZ,43150977-9080-41E0-B75D-B126248B11B9
2,0005D9BD-0247-4F02-B7EE-7C1B44825FA1,0.0,F,1952.0,MI,60CDC115-561D-40C7-98D3-7913BA2607A9
3,00110ABC-DAB9-49E3-A1C8-88BBF8D58109,0.0,M,1970.0,CA,6F6700B8-7D1F-4F1C-9B91-3474148B03A7
4,002667F4-B9A3-4DE2-875F-8034DD637865,0.0,F,1928.0,MD,A088B59F-EBE3-4325-97E0-79E2D83C6BC8
5,0029BBC8-7C22-4444-9F44-87BEF05FE033,0.0,F,1953.0,ID,390E34D8-8DE8-4790-9C33-FE918ABBD080


In [19]:
duplicates = df_train_sp_tot.duplicated()
print(duplicates.sum())

0


#### SyncPatient Origin

In [20]:
df_train_spo.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid'],
      dtype='object')

In [21]:
df_train_spo.shape

(9871, 6)

In [22]:
df_train_spo_tot = df_train_spo.groupby("PatientGuid").agg(
    DMIndicator=("DMIndicator", lambda x: x.mode().iloc[0] if not x.mode().empty else None), 
    Gender=("Gender", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    YearOfBirth=("YearOfBirth", lambda x: x.mode().iloc[0] if not x.mode().empty else None), 
    State=("State", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    PracticeGuid=("PracticeGuid", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [23]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_train_spo_tot,how='left')
df_train_sp_tot.shape

(6600, 6)

In [24]:
print(df_train_sp_tot['PatientGuid'].unique())

['00023761-9D8D-445B-874C-2424CC7CF620'
 '0005D9BD-0247-4F02-B7EE-7C1B44825FA1'
 '00110ABC-DAB9-49E3-A1C8-88BBF8D58109' ...
 'FFEF293B-6147-47F0-8F0B-76180C124205'
 'FFF2B406-9C3F-4A8B-8B27-7179B7D7FBC6'
 'FFF5C634-1CF7-40D5-BDF1-4FA2D62D9C63']


### Primary Keys

In [25]:
primary_keys = [
    "PatientGuid", "ConditionGuid", "TranscriptGuid", 
    "LabPanelGuid", "LabObservationGuid", "MedicationGuid", 
    "DiagnosisGuid", "SmokingStatusGuid"
]

***Reference Date***

In [26]:
reference_date = pd.to_datetime("2012-12-31")

### Table Exploration

***Allergy***

In [27]:
df_train_sa.columns

Index(['AllergyGuid', 'PatientGuid', 'AllergyType', 'StartYear',
       'ReactionName', 'SeverityName', 'MedicationNDCCode', 'MedicationName',
       'UserGuid'],
      dtype='object')

In [28]:
keep_columns_sa = [
    'PatientGuid',         # Patient identifier
    'MedicationNDCCode',   # Medication identifier (T2D comorbidities)
    'MedicationName',      # Name of the medication (T2D comorbidities)
    'AllergyGuid',         # Allergy identifier 
    'AllergyType',         # Type of allergy (could be related to inflammation affecting T2D)
]

df_allergy_new = df_train_sa[keep_columns_sa]
df_allergy_new.head()

,PatientGuid,MedicationNDCCode,MedicationName,AllergyGuid,AllergyType
0,43CE4977-3EC1-48D5-8FCE-8F27BBDDFC1C,7.443429e+07,TriCor (fenofibrate) oral capsule,FFD9BE48-3AD6-40CB-A5CD-001CDCE49A26,Medication
1,622830A1-BA4A-4272-B38B-4C6A9D6F973D,1.780085e+08,Ferrous Fumarate oral tablet,5C0F88CB-1B5A-4EA7-8921-00D777F82918,Medication
2,44BFA67A-9E8E-4447-BFCC-17FF76ADFE21,6.811503e+10,"Norflex (orphenadrine) oral tablet, extended r...",D536315B-A551-4CA5-A505-00E553EFCD4D,Medication
3,74F39AC1-16C5-4A9C-BF59-67513C342070,4.562005e+08,Lexapro (escitalopram) oral tablet,C0A19E94-FEC2-45C6-9D46-00EBD8F2226C,Medication
4,DCFBC6A1-2EBD-41C8-8B17-64D3EC20C082,5.801601e+10,Vicodin (acetaminophen-HYDROcodone) oral tablet,D7779B6B-2FFF-4A87-8F74-00F000127A8E,Medication


In [29]:
df_allergy_tot = df_allergy_new.groupby("PatientGuid").agg(
    MostCommonMedication=("MedicationName", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MedicationNDCCount=("MedicationNDCCode", "count"),  
    MostAllergyTypes=("AllergyType", lambda x: x.mode().iloc[0] if not x.mode().empty else None)
).reset_index()

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4278961325.py:2: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostCommonMedication=("MedicationName", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4278961325.py:2: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostCommonMedication=("MedicationName", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4278961325.py:2: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  MostCommonMedication=("MedicationName", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4278961325.py:2: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  MostCommonMedication=("MedicationName", lambda x: x.mode().iloc[0] if

In [30]:
df_allergy_new.shape

(2797, 5)

In [31]:
df_allergy_tot.shape

(1724, 4)

Merging with train and checking shape

In [32]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_allergy_tot,how='left')
df_train_sp_tot.shape

(6600, 9)

***Diagnosis***

In [33]:
df_train_sd.columns

Index(['DiagnosisGuid', 'PatientGuid', 'ICD9Code', 'DiagnosisDescription',
       'StartYear', 'StopYear', 'Acute', 'UserGuid'],
      dtype='object')

In [34]:
keep_columns_sd = [
    'PatientGuid',         # Patient identifier 
    'DiagnosisGuid',       # Diagnosis identifier (linking)
    'ICD9Code',           # Diagnosis code (comorbid conditions)
    'DiagnosisDescription',# Diagnosis description (understanding the condition)
    'StartYear',           # Year the diagnosis was made (tracking chronic conditions)
    'Acute',               # Acute condition flag (filter out acute conditions)
]

df_diagnosis_new = df_train_sd[keep_columns_sd]
df_diagnosis_new.head()

,PatientGuid,DiagnosisGuid,ICD9Code,DiagnosisDescription,StartYear,Acute
0,BA954BD7-5EE7-4CCF-AB31-07B9C7F72D07,D8873CD9-C72E-4ABD-B0C5-000101A9A72A,825.0,"Fracture of calcaneus, closed",2012,0
1,F8D890EA-7920-40AF-BFCF-42010E1BF563,70F10549-2A75-44F9-8640-00010B10EFAD,784.0,Headache,0,1
2,A3AD2D57-5589-47E2-BDB2-1D5B51764896,A5F5AF96-0B64-472E-BB67-00012544228C,461.9,"Acute sinusitis, unspecified",2010,1
3,2AC1DCDA-9C11-44ED-A2BE-4B1F2393B68E,3C4B7780-6742-4DB5-8DFE-000157ED4DD3,V72.31,Routine gynecological examination,2009,0
4,9DB2B66C-A696-4308-BFA1-4C8F6E97977E,903825EF-AEDD-4F2D-985C-0004D6D95E9B,345.90,"Epilepsy, unspecified, without mention of intr...",2010,0


In [35]:
df_diagnosis_new.loc[:, 'T2D_Diagnosis_Flag'] = df_diagnosis_new.apply(
    lambda row: 1 if (row['Acute'] == 0 and (str(row['ICD9Code']).startswith('249') or str(row['ICD9Code']).startswith('250'))) else 0,
    axis=1
)

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\1766932282.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diagnosis_new.loc[:, 'T2D_Diagnosis_Flag'] = df_diagnosis_new.apply(


In [36]:
df_diagnosis_new.head()

,PatientGuid,DiagnosisGuid,ICD9Code,DiagnosisDescription,StartYear,Acute,T2D_Diagnosis_Flag
0,BA954BD7-5EE7-4CCF-AB31-07B9C7F72D07,D8873CD9-C72E-4ABD-B0C5-000101A9A72A,825.0,"Fracture of calcaneus, closed",2012,0,0
1,F8D890EA-7920-40AF-BFCF-42010E1BF563,70F10549-2A75-44F9-8640-00010B10EFAD,784.0,Headache,0,1,0
2,A3AD2D57-5589-47E2-BDB2-1D5B51764896,A5F5AF96-0B64-472E-BB67-00012544228C,461.9,"Acute sinusitis, unspecified",2010,1,0
3,2AC1DCDA-9C11-44ED-A2BE-4B1F2393B68E,3C4B7780-6742-4DB5-8DFE-000157ED4DD3,V72.31,Routine gynecological examination,2009,0,0
4,9DB2B66C-A696-4308-BFA1-4C8F6E97977E,903825EF-AEDD-4F2D-985C-0004D6D95E9B,345.90,"Epilepsy, unspecified, without mention of intr...",2010,0,0


In [37]:
df_diagnosis_tot = df_diagnosis_new.groupby('PatientGuid').agg(
    T2D_Diagnosis_Flag_Mode=('T2D_Diagnosis_Flag', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostDiagnosisDescription=('DiagnosisDescription', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostCommonICD9=('ICD9Code', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostStartYear=('StartYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostDiagnosisGuid=('DiagnosisGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [38]:
df_diagnosis_new.shape

(94830, 7)

In [39]:
df_diagnosis_tot.shape

(9948, 6)

Merging with train and checking shape

In [40]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_diagnosis_tot,how='left')
df_train_sp_tot.shape

(6600, 14)

### ***Lab***

#### Lab Observation 

In [41]:
df_train_slo.columns

Index(['HL7Identifier', 'HL7Text', 'LabObservationGuid', 'LabPanelGuid',
       'HL7CodingSystem', 'ObservationValue', 'Units', 'ReferenceRange',
       'AbnormalFlags', 'ResultStatus', 'ObservationYear', 'UserGuid',
       'IsAbnormalValue'],
      dtype='object')

In [42]:
df_train_slo.head()

,HL7Identifier,HL7Text,LabObservationGuid,LabPanelGuid,HL7CodingSystem,ObservationValue,Units,ReferenceRange,AbnormalFlags,ResultStatus,ObservationYear,UserGuid,IsAbnormalValue
0,001016,"Calcium, Serum",30F3917A-E4DE-4ADD-BE2A-6E842B1CF958,AC8B2E6D-E55E-42C5-B0F4-099094703A4B,LabCorpLocal,9.8,mg/dL,8.6-10.2,0,Final,2011,3027FCA9-C076-4B4D-823E-840FE03E1AFE,0
1,001016,"Calcium, Serum",29C33291-2E32-46E1-9FDC-4D73783867E6,B8A16544-6185-43E7-ADE1-258E4781CCC7,LabCorpLocal,9.6,mg/dL,8.7-10.2,0,Final,2011,00000000-0000-0000-0000-000000000000,0
2,001016,"Calcium, Serum",06586D95-D7F3-4E63-B8AA-5751F3213D98,C9E3816D-C533-4046-AD79-B4EC5CB7DEFA,LabCorpLocal,10.1,mg/dL,8.6-10.2,0,Final,2011,B366A9CC-814C-4B2F-99E5-4DC5C79B9DEC,0
3,001016,"Calcium, Serum",2ADEC63D-D741-48F5-9024-B103F2BBCC3B,D17E087C-7783-42E8-A875-E4870C2F5EC0,LabCorpLocal,9.5,mg/dL,8.6-10.2,0,Final,2011,60C80223-5A28-4E74-89FF-73505D39BCDD,0
4,001016,"Calcium, Serum",CCB87D9F-DCF8-4145-B4A8-5082F9E2FDA7,A1455081-17F7-4714-89E3-CACE4BA5BBC5,LabCorpLocal,9.2,mg/dL,8.7-10.2,0,Final,2011,00000000-0000-0000-0000-000000000000,0


In [43]:
keep_columns_lo = [
    'LabPanelGuid',
    'LabObservationGuid',
    'HL7Identifier',
    'HL7Text',
    'ObservationValue',
    'Units',
    'AbnormalFlags',
    'IsAbnormalValue',
]

df_slo_new = df_train_slo[keep_columns_lo]
df_slo_new.head()

,LabPanelGuid,LabObservationGuid,HL7Identifier,HL7Text,ObservationValue,Units,AbnormalFlags,IsAbnormalValue
0,AC8B2E6D-E55E-42C5-B0F4-099094703A4B,30F3917A-E4DE-4ADD-BE2A-6E842B1CF958,001016,"Calcium, Serum",9.8,mg/dL,0,0
1,B8A16544-6185-43E7-ADE1-258E4781CCC7,29C33291-2E32-46E1-9FDC-4D73783867E6,001016,"Calcium, Serum",9.6,mg/dL,0,0
2,C9E3816D-C533-4046-AD79-B4EC5CB7DEFA,06586D95-D7F3-4E63-B8AA-5751F3213D98,001016,"Calcium, Serum",10.1,mg/dL,0,0
3,D17E087C-7783-42E8-A875-E4870C2F5EC0,2ADEC63D-D741-48F5-9024-B103F2BBCC3B,001016,"Calcium, Serum",9.5,mg/dL,0,0
4,A1455081-17F7-4714-89E3-CACE4BA5BBC5,CCB87D9F-DCF8-4145-B4A8-5082F9E2FDA7,001016,"Calcium, Serum",9.2,mg/dL,0,0


In [44]:
unique_hl7 = df_slo_new['HL7Identifier'].unique()
print("Unique HL7Identifier Values:")
print(unique_hl7)

unique_hl7_text = df_slo_new['HL7Text'].unique()
print("Unique HL7 Text:")
print(sorted(unique_hl7_text))

Unique HL7Identifier Values:
['001016' '001040' '001057' '001065' '001073' '001081' '001099' '001107'
 '001123' '001149' '001180' '001198' '001206' '001222' '001315' '001339'
 '001347' '001348' '001362' '001370' '001404' '001503' '001537' '001545'
 '001578' '001725' '001784' '002019' '002030' '004227' '004261' '004264'
 '004598' '005025' '005033' '005041' '005058' '005076' '005077' '005215'
 '006072' '006201' '006510' '006627' '006677' '007386' '008847' '010334'
 '010389' '011150' '011247' '011362' '011577' '011849' '011916' '011976'
 '012039' '012047' '012054' '012237' '012581' '012684' '012692' '013045'
 '013052' '013060' '013078' '013094' '013102' '013104' '013105' '013106'
 '013110' '013128' '013136' '013145' '013146' '013147' '013148' '013152'
 '013185' '013672' '015065' '015073' '015081' '015107' '015108' '015123'
 '015131' '015149' '015156' '015172' '015180' '015255' '015289' '015610'
 '015909' '015911' '015917' '015925' '015933' '015941' '015945' '016014'
 '016055' '016063' '01

In [45]:
t2d_hl7_identifiers = [
    '18282-4', '1558-6', '20405-7', '2075-0', '2345-7',  # Glucose Monitoring
    '4548-4',  # HbA1c
    '2093-3', '2571-8', '2823-3', '2085-9'  # Lipid Profile
]

In [46]:
unique_av = df_slo_new['IsAbnormalValue'].unique()
print("Unique Abnormal Values:")
print(unique_av)

Unique Abnormal Values:
[0 1]


In [47]:
unique_av_values = df_slo_new['AbnormalFlags'].unique()
print("Unique Abnormal Flags:")
print(unique_av_values)

Unique Abnormal Flags:
[0 'Below Normal Low' 'Above Normal High' 'Panic High' 'Abnormal Result'
 'Alert High' 'Panic Low' 'Alert Low' 'UKNOWN']


In [48]:
df_slo_new.loc[:, 'T2D_Lab_Observation_Flag'] = df_slo_new.apply(
    lambda row: 1 if (row['HL7Identifier'] in t2d_hl7_identifiers and row['IsAbnormalValue'] == 1) else 0,
    axis=1
)

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\908909369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slo_new.loc[:, 'T2D_Lab_Observation_Flag'] = df_slo_new.apply(


In [49]:
df_slo_new.head()

,LabPanelGuid,LabObservationGuid,HL7Identifier,HL7Text,ObservationValue,Units,AbnormalFlags,IsAbnormalValue,T2D_Lab_Observation_Flag
0,AC8B2E6D-E55E-42C5-B0F4-099094703A4B,30F3917A-E4DE-4ADD-BE2A-6E842B1CF958,001016,"Calcium, Serum",9.8,mg/dL,0,0,0
1,B8A16544-6185-43E7-ADE1-258E4781CCC7,29C33291-2E32-46E1-9FDC-4D73783867E6,001016,"Calcium, Serum",9.6,mg/dL,0,0,0
2,C9E3816D-C533-4046-AD79-B4EC5CB7DEFA,06586D95-D7F3-4E63-B8AA-5751F3213D98,001016,"Calcium, Serum",10.1,mg/dL,0,0,0
3,D17E087C-7783-42E8-A875-E4870C2F5EC0,2ADEC63D-D741-48F5-9024-B103F2BBCC3B,001016,"Calcium, Serum",9.5,mg/dL,0,0,0
4,A1455081-17F7-4714-89E3-CACE4BA5BBC5,CCB87D9F-DCF8-4145-B4A8-5082F9E2FDA7,001016,"Calcium, Serum",9.2,mg/dL,0,0,0


In [50]:
df_lo_tot = df_slo_new.groupby('LabPanelGuid').agg(
    T2D_Lab_Observation_Mode=('T2D_Lab_Observation_Flag', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostHL7Identifier=('HL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    AvgObservationValue=("ObservationValue", "mean"),
    MostAbnormalFlags=('AbnormalFlags', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostIsAbnormalValue=('IsAbnormalValue', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2494931097.py:3: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostHL7Identifier=('HL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2494931097.py:3: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostHL7Identifier=('HL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2494931097.py:3: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostHL7Identifier=('HL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2494931097.py:3: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostHL7Identifier=('HL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().em

In [51]:
df_slo_new.shape

(29014, 9)

In [52]:
df_lo_tot.shape

(6579, 6)

#### Lab Panel

In [53]:
df_train_slp.columns

Index(['PanelName', 'LabPanelGuid', 'LabResultGuid', 'ObservationYear',
       'Status'],
      dtype='object')

In [54]:
unique_pn = df_train_slp['PanelName'].unique()
print("Unique Panel Name Values:")
print(unique_pn)

unique_lr = df_train_slp['LabResultGuid'].unique()
print("Unique Lab Result ID Values:")
print(unique_lr)

unique_os = df_train_slp['ObservationYear'].unique()
print("Unique ObservationYear Values:")
print(unique_os)

Unique Panel Name Values:
['083824+CBC/D/Plt+RPR+Rh+ABO...' '726778 7+Alc-Unbund'
 '764888 10+Oxyco+Alc+Crt-Bund' '794388 7 Drug-Bund'
 'Actin (Smooth Muscle) Antibody' 'Aerobic Bacterial Culture' 'AFP Tetra'
 'AFP, Serum, Tumor Marker' 'Albumin, Serum' 'Aldolase'
 'Aldosterone LCMS, Serum' 'Alkaline Phosphatase'
 'Alkaline Phosphatase, S' 'Allergen Profile, Food-Basic'
 'Allergen, Alder Tree' 'Allergen, Alternaria tenuis'
 'Allergen, Aspergillus fumigatus' 'Allergen, Bermuda Grass'
 'Allergen, Birch' 'Allergen, Cat Dander and epithelium'
 'Allergen, Cladosporium herba.' 'Allergen, Clam'
 'Allergen, Codfish/Whitefish' 'Allergen, Common ragweed' 'Allergen, Corn'
 'Allergen, D. farinae' 'Allergen, D. pteronyssinus'
 'Allergen, Dog Dander' 'Allergen, Egg White' 'Allergen, Elm Tree'
 'Allergen, German Cockroach' 'Allergen, Milk(Cow)'
 'Allergen, Mountain Cedar/Juniper' 'Allergen, Mugwort'
 'Allergen, Mulberry Tree' 'Allergen, Oak Tree' 'Allergen, Olive Tree'
 'Allergen, Peanut' 'Allergen, 

In [55]:
t2d_tests = [
    'Hemoglobin A1c', 'Glucose Tolerance (3 Sp Blood)', 'Glucose, Plasma', 'Glucose, Serum', 'Insulin',
    'C-Peptide, Serum', 'Gest. Diabetes 1-Hr Screen', 'Gestational 2 hour GTT', 'Gestational Glucose Tolerance'
]

In [56]:
df_train_slp_new=df_train_slp

In [57]:
df_train_slp_new.loc[:, 'T2D_Test_Flag'] = df_train_slp['PanelName'].apply(
    lambda x: 1 if any(test in str(x) for test in t2d_tests) else 0
)

In [58]:
df_train_slp_new.head()

,PanelName,LabPanelGuid,LabResultGuid,ObservationYear,Status,T2D_Test_Flag
0,083824+CBC/D/Plt+RPR+Rh+ABO...,356133F9-70AD-4C03-9FC2-5B134C84D3D2,E2E9F32D-4277-4078-A466-CF9714718D56,2011,Final,0
1,083824+CBC/D/Plt+RPR+Rh+ABO...,3B90AE78-024D-4183-90D4-9592B638554A,938556AC-E98D-4386-A94A-C89A83FA570C,2012,Final,0
2,083824+CBC/D/Plt+RPR+Rh+ABO...,A7ED264E-A0A9-41E0-8D1D-0A65CC1BF927,34CE5F46-0ED5-44B7-B004-771EFA0EDE81,2011,Final,0
3,083824+CBC/D/Plt+RPR+Rh+ABO...,7958A75D-A5AF-4C10-AC2D-9F1AFD5758B9,840ED500-8FC6-468A-B7AF-B429CC3FDD7C,2011,Final,0
4,083824+CBC/D/Plt+RPR+Rh+ABO...,EA8BA390-2454-4905-8437-121FD48DF555,642AEA75-D604-45B3-94DF-9AC4F1C1AEEA,2012,Final,0


In [59]:
df_slp_tot = df_train_slp_new.groupby('LabPanelGuid').agg(
    T2D_Test_Mode=('T2D_Test_Flag', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostLabResultGuid=('LabResultGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostObservationYear=('ObservationYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostStatus=('Status', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [60]:
df_train_slp_new.shape

(11139, 6)

In [61]:
df_slp_tot.shape

(11139, 5)

#### **Merging Lab Observation & Panel**

In [62]:
df_lo_lp = df_lo_tot.merge(df_slp_tot, how='left')

In [63]:
df_lo_lp.shape

(6579, 10)

In [64]:
df_lo_lp.head()

,LabPanelGuid,T2D_Lab_Observation_Mode,MostHL7Identifier,AvgObservationValue,MostAbnormalFlags,MostIsAbnormalValue,T2D_Test_Mode,MostLabResultGuid,MostObservationYear,MostStatus
0,00085E99-3253-43B3-A94B-CFD4AADF7EB7,0,2571-8,122.000000,0,0,0,194AB2E4-440E-49B6-92D2-E86244173BBF,2011,Final
1,000DF53B-2A3F-4C39-A273-BB06D7FCFA60,0,10834-0,19.883333,0,0,0,54614552-1387-4F31-BE20-6FD707B13D48,2011,Final
2,00133D23-40D0-4CAF-8C0C-65408711E733,0,2571-8,91.000000,0,0,0,6E6B3457-BAE8-46E8-B47B-5529AC2BC8E1,2012,Final
3,00135BF6-35B3-4800-929C-E5D34DAA9A37,0,2571-8,121.000000,0,0,0,C793E33C-AECE-4BA6-BF17-D8A885DB842B,2012,Final
4,0013DE1C-22D2-4918-9089-C0D18781BCAA,0,20405-7,0.944444,0,0,0,D101D296-BA02-4C9E-AC19-DE2D0BDB719E,2011,Final


#### Lab Result

In [65]:
df_train_slr.columns

Index(['LabResultGuid', 'UserGuid', 'PatientGuid', 'TranscriptGuid',
       'PracticeGuid', 'FacilityGuid', 'ReportYear', 'AncestorLabResultGuid'],
      dtype='object')

In [66]:
keep_columns_lr = [
    'LabResultGuid',
    'PatientGuid',
    'TranscriptGuid',
    'ReportYear',
    'AncestorLabResultGuid',
]

df_train_slr_new = df_train_slr[keep_columns_lr]
df_train_slr_new.head()

,LabResultGuid,PatientGuid,TranscriptGuid,ReportYear,AncestorLabResultGuid
0,9B66DC54-51BA-4851-BE1F-007E994839B9,ECD01885-2FB6-41D2-9220-286F67F57282,00000000-0000-0000-0000-000000000000,2011,00000000-0000-0000-0000-000000000000
1,E2206C79-B879-4BD7-BE8F-00A00A29751F,ED48DCBA-1B6A-4A99-B17E-15537EFB137A,00000000-0000-0000-0000-000000000000,2011,00000000-0000-0000-0000-000000000000
2,F7078302-BCBF-4626-B0B7-00B105577727,D6F957FF-C1F4-4B52-8481-A543C5C20E58,00000000-0000-0000-0000-000000000000,2012,00000000-0000-0000-0000-000000000000
3,F9099B02-ACE7-4943-A586-00D5EEBA196A,FF1059B3-2B7A-4161-AF23-C3B07FB8BB92,00000000-0000-0000-0000-000000000000,2011,00000000-0000-0000-0000-000000000000
4,7A8AD63C-200E-4AE5-93AE-0108498EE481,4651E19F-9DDE-4AF6-90C4-FEEDFAF10643,00000000-0000-0000-0000-000000000000,2011,00000000-0000-0000-0000-000000000000


In [67]:
unique_lr = df_train_slr_new['LabResultGuid'].unique()
print("Unique Lab Result Values:")
print(unique_lr)

Unique Lab Result Values:
['9B66DC54-51BA-4851-BE1F-007E994839B9'
 'E2206C79-B879-4BD7-BE8F-00A00A29751F'
 'F7078302-BCBF-4626-B0B7-00B105577727' ...
 '4DACA8EE-FB9D-4AA4-AD15-FF211254323C'
 '59FD913A-2D24-4910-AE8F-FFC7AC6AB1A8'
 'DA63D9B2-4B88-4912-B16A-FFD6E712CAD5']


In [68]:
df_slr_tot = df_train_slr_new.groupby('LabResultGuid').agg(
    MostPatientGuid=('PatientGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostReportYear=('ReportYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostAncestorLabResultGuid=('AncestorLabResultGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [69]:
df_train_slr_new.shape

(1952, 5)

In [70]:
df_slr_tot.shape

(1952, 4)

#### **Merging Lab Observation & Panel & Result**

In [71]:
df_lo_lp_lr = df_lo_lp.merge(df_slr_tot, how='left',left_on='MostLabResultGuid', right_on='LabResultGuid')

In [72]:
df_lo_lp_lr.shape

(6579, 14)

In [73]:
df_lo_lp_lr.rename(columns={'MostPatientGuid': 'PatientGuid'}, inplace=True)

In [74]:
df_lab_tot = df_lo_lp_lr.groupby('PatientGuid').agg(
    MostPanelID=('LabPanelGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostLabObservation=('T2D_Lab_Observation_Mode', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostHL7Identifier=('MostHL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostAbnormalFlags=('MostAbnormalFlags', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostIsAbnormalValue=('MostIsAbnormalValue', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostStatus=('MostStatus', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostLabTest=('T2D_Test_Mode', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostLabResultGuid=('MostLabResultGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostReportYear=('MostReportYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostAncestorLabResultGuid=('MostAncestorLabResultGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\3484641597.py:4: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  MostHL7Identifier=('MostHL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\3484641597.py:4: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  MostHL7Identifier=('MostHL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\3484641597.py:4: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostHL7Identifier=('MostHL7Identifier', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\3484641597.py:4: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostHL7Identifier=('MostHL7Identifier', lambda x: x.mode().iloc[0] if

In [75]:
df_lab_tot.shape

(791, 11)

Merging with train and checking shape.

In [76]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_lab_tot,how='left')
df_train_sp_tot.shape

(6600, 24)

### ***Condition***

#### Condition

In [77]:
df_sc.columns

Index(['ConditionGuid', 'Code', 'Name'], dtype='object')

In [78]:
print(df_sc['Name'].unique())
print(df_sc['Code'].unique())
print(df_sc['ConditionGuid'].unique())

['No Known Diagnoses' 'No Known Allergies' 'No Known Medications']
['NOKNOWNDIAGNOSES' 'NOKNOWNALLERGIES' 'NOKNOWNMEDICATIONS']
['D4FF24EA-658D-4500-A71F-821F9776FB27'
 '4BF4503F-4EA7-4E63-BBA6-B6A4A8C273F8'
 '2AE517F6-5029-4BC4-9353-FB8E0CC3A822']


In [79]:
df_sc.head()

,ConditionGuid,Code,Name
0,D4FF24EA-658D-4500-A71F-821F9776FB27,NOKNOWNDIAGNOSES,No Known Diagnoses
1,4BF4503F-4EA7-4E63-BBA6-B6A4A8C273F8,NOKNOWNALLERGIES,No Known Allergies
2,2AE517F6-5029-4BC4-9353-FB8E0CC3A822,NOKNOWNMEDICATIONS,No Known Medications


#### Patient Condition

In [80]:
df_train_spc.columns

Index(['PatientConditionGuid', 'PatientGuid', 'ConditionGuid', 'CreatedYear'], dtype='object')

In [81]:
df_train_spc.shape

(2836, 4)

In [82]:
unique_pc = df_train_spc['PatientConditionGuid'].unique()
print("Unique Patient Condition Values:")
print(unique_pc)

unique_cg = df_train_spc['ConditionGuid'].unique()
print("Unique Condition Values:")
print(unique_cg)

Unique Patient Condition Values:
['22A5A7B5-FF9F-4FE6-86B2-00314970CF94'
 'F9BE2F6F-87DD-4816-9ADE-00423A35C7DB'
 '61A844C2-01B0-4DBF-B4AE-0062C30CD9B0' ...
 '5A4D233B-774E-4A89-A351-FFA62BF8DA52'
 '4BF94ED3-DA4D-4DA4-9C18-FFA7C031743B'
 'C0FD40F3-25CD-44FD-8570-FFEB0341FEA2']
Unique Condition Values:
['4BF4503F-4EA7-4E63-BBA6-B6A4A8C273F8'
 '2AE517F6-5029-4BC4-9353-FB8E0CC3A822']


#### **Merging Condition & Patient Condition**

In [83]:
df_sc_spc = df_sc.merge(df_train_spc, how='left')

In [84]:
df_sc_spc.shape

(2837, 6)

In [85]:
df_sc_spc_tot = df_sc_spc.groupby('ConditionGuid').agg(
    MostPatientGuid=('PatientGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostPatientConditionGuid=('PatientConditionGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostCreatedYear=('CreatedYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [86]:
df_sc_spc_tot.shape

(3, 4)

In [87]:
df_sc_spc_tot.rename(columns={'MostPatientGuid': 'PatientGuid'}, inplace=True)

Merging with train and checking shape.

In [88]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_sc_spc_tot,how='left')
df_train_sp_tot.shape

(6600, 27)

***Medication***

In [89]:
df_train_sm.columns

Index(['MedicationGuid', 'PatientGuid', 'NdcCode', 'MedicationName',
       'MedicationStrength', 'Schedule', 'DiagnosisGuid', 'UserGuid'],
      dtype='object')

In [90]:
keep_columns_sm = [
    'PatientGuid',         # Patient identifier 
    'MedicationGuid',      
    'DiagnosisGuid',           # Diagnosis code (comorbid conditions)
    'NdcCode',
    'MedicationName',           # Year the diagnosis was made (tracking chronic conditions)
]

df_train_sm_new = df_train_sm[keep_columns_sm]
df_train_sm_new.head()

,PatientGuid,MedicationGuid,DiagnosisGuid,NdcCode,MedicationName
0,A7F1591C-9DB8-4F12-A0B8-15AC12C2B801,0D6B339C-AB1F-4FF1-B231-000026C414C9,DA833B28-CE04-4547-A94F-8A4E6C187A3B,23490544301,Digoxin oral tablet
1,7A88D55F-22A9-4126-861D-92F0DA2ED701,72ADD073-CDA0-4F85-B6CB-00008B645E90,3ED5B90C-22A1-46EE-9CF5-027E684627D8,143126730,Lisinopril oral tablet
2,F0B80C93-D371-44BE-B464-530C97FE6C02,D35E505A-02F8-4EFD-A460-00042F85EF3E,0B6C3285-1D8C-4C0F-9928-95EC226CB2FE,456069801,Tessalon (benzonatate) oral capsule
3,FF32CBDA-DDB1-4730-81C5-ACB0F321C2C5,FFB036B9-A636-48F2-967C-0005F8D78574,DE2E15A5-5080-4141-A061-C81EC739C0A4,60432060504,Promethazine VC Plain (phenylephrine-promethaz...
4,1ECD93C8-75BD-4D3C-8D15-E2A3D0373090,AB095685-A703-4D81-8678-0009396BF105,16B189A4-359C-41E1-BAA3-3E081EA24C41,247211730,Lexapro (escitalopram) oral tablet


In [91]:
print(df_train_sm_new['MedicationName'].unique())

['Digoxin oral tablet' 'Lisinopril oral tablet'
 'Tessalon (benzonatate) oral capsule' ...
 'Benzac W (benzoyl peroxide topical) topical liquid'
 'Allegra (fexofenadine) oral suspension'
 'Metryl (metroNIDAZOLE) oral tablet']


In [92]:
df_med_tot = df_train_sm_new.groupby('PatientGuid').agg(
    MostMedicationGuid=('MedicationGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MedicationNDCCount=("NdcCode", "count"), 
    MostDiagnosisGuid=('DiagnosisGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostCommonMedication=('MedicationName', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\1189344984.py:5: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostCommonMedication=('MedicationName', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\1189344984.py:5: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostCommonMedication=('MedicationName', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\1189344984.py:5: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostCommonMedication=('MedicationName', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\1189344984.py:5: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  MostCommonMedication=('MedicationName', lambda x: x.mode().iloc[0] if

In [93]:
df_med_tot.shape

(9846, 5)

Merging with train and checking shape.

In [94]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_med_tot,how='left')
df_train_sp_tot.shape

(6600, 28)

***Prescription***

In [95]:
df_train_spr.columns

Index(['PrescriptionGuid', 'PatientGuid', 'MedicationGuid', 'PrescriptionYear',
       'Quantity', 'NumberOfRefills', 'RefillAsNeeded', 'GenericAllowed',
       'UserGuid'],
      dtype='object')

In [96]:
keep_columns_spr = [
    'PatientGuid',         
    'PrescriptionGuid',      
    'MedicationGuid',          
    'PrescriptionYear',       
]

df_train_spr_new = df_train_spr[keep_columns_spr]
df_train_spr_new.head()

,PatientGuid,PrescriptionGuid,MedicationGuid,PrescriptionYear
0,C907DB37-6FFE-4102-B12D-CCE965A91AB6,DD22B641-A6BA-4B0C-957A-00012731F571,81378082-ED3B-4FB4-AADC-E16F96E3FABA,2010
1,35F5194C-22FE-4960-BAF6-15E616D67B66,736B99DB-34A2-4A08-9760-000201B2D7C7,0873ABE3-E104-4BB3-8416-8C9E1C8912C1,2009
2,A45C5F8F-030A-421A-BC07-669FDE202D30,2008B573-370C-443E-8486-000201BCFAA6,A0AD4801-7601-4D18-A623-A906DCF42C7E,2010
3,DC5B1E3B-E789-4BF2-AE36-71465165EF56,C7D62A47-B33B-48CE-9C5A-00022DB765C5,BAC517F0-03E3-4E03-BD7F-0B626E496437,2011
4,24E8D498-65BE-4F9E-A9FE-6BACD50E5C57,274181D8-7927-4D64-B137-0002E45116CD,C73551FD-A00E-406B-BDE0-4491802954CC,2010


In [97]:
df_prescription_tot = df_train_spr_new.groupby('PatientGuid').agg(
    MostPrescriptionGuid=('PrescriptionGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostMedicationGuid=('MedicationGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostPrescriptionYear=('PrescriptionYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [98]:
df_prescription_tot.shape

(8953, 4)

Merging with train and checking shape.

In [99]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_prescription_tot,how='left')
df_train_sp_tot.shape

(6600, 30)

### ***Smoking***

#### Smoking Status

In [100]:
df_sss.columns

Index(['SmokingStatusGuid', 'Description', 'NISTcode'], dtype='object')

In [101]:
df_sss.head()

,SmokingStatusGuid,Description,NISTcode
0,E86CA3A8-E35B-4BBF-80E2-0375AB4A1460,Current status unknown,9
1,1F3BFBBF-AB76-481B-B1E0-08A3689A54BC,Not a current tobacco user,0
2,C12C2DB7-D31A-4514-88C0-42CBD339F764,0 cigarettes per day (previous smoker),3
3,FA2B7AE4-4D14-4768-A8C7-55B5F0CDF4AF,Few (1-3) cigarettes per day,2
4,0815F240-3DD3-43C6-8618-613CA9E41F9F,Current status unknown,5


In [102]:
unique_NIST = df_sss['NISTcode'].unique()
print(unique_NIST)

unique_description = df_sss['Description'].unique()
print(unique_description)

[9 0 3 2 5 1 4]
['Current status unknown' 'Not a current tobacco user'
 '0 cigarettes per day (previous smoker)' 'Few (1-3) cigarettes per day'
 '2 or more packs per day' 'Up to 1 pack per day' '1-2 packs per day'
 '0 cigarettes per day (non-smoker or less than 100 in lifetime)'
 'Current Tobacco user']


In [103]:
df_sss.loc[:, 'T2D_Smoking_Flag'] = df_sss.apply(
    lambda row: 1 if row['Description'] in ['Current Tobacco user', 'Few (1-3) cigarettes per day', 'Up to 1 pack per day', 
                                              '1-2 packs per day', '2 or more packs per day'] else 0,
    axis=1
)

In [104]:
df_sss.head()

,SmokingStatusGuid,Description,NISTcode,T2D_Smoking_Flag
0,E86CA3A8-E35B-4BBF-80E2-0375AB4A1460,Current status unknown,9,0
1,1F3BFBBF-AB76-481B-B1E0-08A3689A54BC,Not a current tobacco user,0,0
2,C12C2DB7-D31A-4514-88C0-42CBD339F764,0 cigarettes per day (previous smoker),3,0
3,FA2B7AE4-4D14-4768-A8C7-55B5F0CDF4AF,Few (1-3) cigarettes per day,2,1
4,0815F240-3DD3-43C6-8618-613CA9E41F9F,Current status unknown,5,0


In [105]:
df_sss_tot = df_sss.groupby('SmokingStatusGuid').agg(
    MostDescription=('Description', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostNISTCode=('NISTcode', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostT2DSmokingFlag=('T2D_Smoking_Flag', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [106]:
df_sss_tot.shape

(10, 4)

#### Patient Smoking Status

In [107]:
df_train_spss.columns

Index(['PatientSmokingStatusGuid', 'PatientGuid', 'SmokingStatusGuid',
       'EffectiveYear'],
      dtype='object')

In [108]:
keep_columns_spss = [
    'PatientGuid',         
    'SmokingStatusGuid',            
]

df_train_spss_new = df_train_spss[keep_columns_spss]
df_train_spss_new.head()

,PatientGuid,SmokingStatusGuid
0,418B3B82-D178-47D3-A6D4-B9F92A37D7DC,FCD437AA-0451-4D8A-9396-B6F19D8B25E8
1,53183B47-C9F6-46D4-ACD1-D2B44B6263B0,DD01E545-D7AF-4F00-B248-9FD40010D81D
2,4E01E2C2-387D-485E-8AFA-312AAB7B5026,1F3BFBBF-AB76-481B-B1E0-08A3689A54BC
3,B732CBF8-E855-4CA3-9548-7C0A1660CC52,5ABBAB35-836F-4F3E-8632-CE063828DA15
4,4AE52DF5-AB5B-4272-AF6C-789EFE0B58B3,5ABBAB35-836F-4F3E-8632-CE063828DA15


#### **Merging Smoking Status & Patient Smoking Status**

In [109]:
df_sss_spss=df_sss_tot.merge(df_train_spss_new,how='left')

In [110]:
df_sss_spss.shape

(4940, 5)

In [111]:
df_sss_spss_tot = df_sss_spss.groupby('SmokingStatusGuid').agg(
    MostPatientGuid=('PatientGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostT2DSmokingFlag=('MostT2DSmokingFlag', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostDescription=('MostDescription', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostNISTCode=('MostNISTCode', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [112]:
df_sss_spss_tot.shape

(10, 5)

In [113]:
df_sss_spss_tot.rename(columns={'MostPatientGuid': 'PatientGuid'}, inplace=True)

Merging with train and checking shape.

In [114]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_sss_spss_tot,how='left')
df_train_sp_tot.shape

(6600, 34)

### ***Transcript***

#### Transcript

In [115]:
df_train_st.columns

Index(['TranscriptGuid', 'PatientGuid', 'VisitYear', 'Height', 'Weight', 'BMI',
       'SystolicBP', 'DiastolicBP', 'RespiratoryRate', 'HeartRate',
       'Temperature', 'PhysicianSpecialty', 'UserGuid'],
      dtype='object')

In [116]:
keep_columns_st = [
    'TranscriptGuid',         
    'PatientGuid',      
    'VisitYear',          
    'Height',
    'Weight',
    'BMI',
    'SystolicBP',
    'DiastolicBP',
    'RespiratoryRate',
    'HeartRate'
]

df_train_st_new = df_train_st[keep_columns_st]
df_train_st_new.head()

,TranscriptGuid,PatientGuid,VisitYear,Height,Weight,BMI,SystolicBP,DiastolicBP,RespiratoryRate,HeartRate
0,4E74D809-F0C6-4A4D-8F6F-0000042B3AEA,7F78C9FD-8A95-48DC-B074-FA41F5B41826,2009,63.0,145.8,25.824,0.0,0.0,0.0,0.0
1,84A5F60E-49F6-4437-BAE1-00004F4D15EE,EE1E2D06-8846-4BF5-9C82-E08BC8798DAE,2011,0.0,0.0,0.000,0.0,0.0,0.0,0.0
2,7C45CC93-4799-42A1-B6D6-00008D1B3579,1FF23603-B9D4-42B0-B450-BCD5EA46A025,2010,64.5,133.0,22.474,106.0,66.0,0.0,0.0
3,375F3C1C-0058-4872-B96E-00009FE9328A,16C2D99F-1494-4DF9-82E6-0FA0F0EB1B63,2011,68.0,125.4,19.065,119.0,64.0,0.0,0.0
4,66439EEB-2FFB-4711-87F2-0001690835A6,18256C20-BAC8-4F57-A03C-2C7152CE1813,2011,0.0,0.0,0.000,0.0,0.0,0.0,0.0


In [117]:
df_st_tot = df_train_st_new.groupby('PatientGuid').agg(
    MostVisitYear=('VisitYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostTranscriptGuid=('TranscriptGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    AvgHeight=('Height', "mean"),
    AvgWeight=('Weight', "mean"),
    AvgBMI=('BMI', "mean"),
    AvgSystolicBP=('SystolicBP', "mean"),
    AvgDiastolicBP=('DiastolicBP', "mean"),
    AvgRespiratoryRate=('RespiratoryRate', "mean"),
    AvgHeartRate=('HeartRate', "mean"),
).reset_index()

In [118]:
df_st_tot.shape

(9948, 10)

Merging with train and checking shape.

In [119]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_st_tot,how='left')
df_train_sp_tot.shape

(6600, 43)

#### Transcript Allergy

In [120]:
df_train_sta.columns

Index(['TranscriptAllergyGuid', 'TranscriptGuid', 'AllergyGuid'], dtype='object')

In [121]:
df_train_sta.shape

(801, 3)

In [122]:
df_train_sta.head()

,TranscriptAllergyGuid,TranscriptGuid,AllergyGuid
0,7871260D-D525-4FBC-B747-00BF63BF565F,9994BCA5-70A2-4AA1-AD74-70C76D2E0ED1,496E9CDC-D811-470A-BBC3-11944A1F33D6
1,17BA575D-8A8C-46E8-B67A-00C9A388D667,FC989B9E-5378-4C32-A7AC-5B65B3743082,51C60FE1-F9E9-4ACA-8C14-CB2342E59E3B
2,0BC7D70E-1566-4E32-8CEE-010CB8AE06FE,B6B7069D-4D4F-42B8-8DD8-2437D90E6ADA,1A14A7C4-7A7D-4D3F-809C-FBEEFCB9A232
3,FD726AA1-8E2E-48D3-A6A5-021E8845B483,AA2E21E0-CBB3-4034-9436-AFEF808F7E1B,D998FA81-9D68-409C-BA72-0BAB54197749
4,6026AB5A-55A0-4294-8C7F-021F7C0D9DFF,0B0898F6-52B3-4E46-948F-6E88A96E427E,B590A7F7-B7A8-475C-9CAC-C5B353B31267


In [123]:
df_st_tot.rename(columns={'MostTranscriptGuid': 'TranscriptGuid'}, inplace=True)

In [124]:
df_st_ta = df_st_tot.merge(df_train_sta, how='left')

In [125]:
df_st_ta.shape

(9957, 12)

#### Transcript Diagnosis

In [126]:
df_train_std.columns

Index(['TranscriptDiagnosisGuid', 'TranscriptGuid', 'DiagnosisGuid'], dtype='object')

In [127]:
df_train_std.shape

(267938, 3)

In [128]:
df_st_ta_td = df_st_ta.merge(df_train_std, how='left')

In [129]:
df_st_ta_td.shape

(24445, 14)

#### Transcript Medication

In [130]:
df_train_stm.columns

Index(['TranscriptMedicationGuid', 'TranscriptGuid', 'MedicationGuid'], dtype='object')

In [131]:
df_st_ta_td_tm = df_st_ta_td.merge(df_train_stm, how='left')

In [132]:
df_st_ta_td_tm.shape

(46390, 16)

In [133]:
df_st_ta_td_tm_tot = df_st_ta_td_tm.groupby('TranscriptGuid').agg(
    MostPatientGuid=('PatientGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostVisitYear=('MostVisitYear', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostTranscriptGuid=('TranscriptGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostTranscriptAllergyGuid=('TranscriptAllergyGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostTranscriptDiagnosisGuid=('TranscriptDiagnosisGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostDiagnosisGuid=('DiagnosisGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostTranscriptMedicationGuid=('TranscriptMedicationGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostAllergyGuid=('AllergyGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    MostMedicationGuid=('MedicationGuid', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    AvgHeight=('AvgHeight', "mean"),
    AvgWeight=('AvgWeight', "mean"),
    AvgBMI=('AvgBMI', "mean"),
    AvgSystolicBP=('AvgSystolicBP', "mean"),
    AvgDiastolicBP=('AvgDiastolicBP', "mean"),
    AvgRespiratoryRate=('AvgRespiratoryRate', "mean"),
    AvgHeartRate=('AvgHeartRate', "mean"),
).reset_index()

In [134]:
df_st_ta_td_tm_tot.shape

(9948, 17)

In [135]:
df_st_ta_td_tm_tot.rename(columns={'MostPatientGuid': 'PatientGuid'}, inplace=True)

Merging with train and checking shape.

In [136]:
df_train_sp_tot=pd.merge(df_train_sp_tot,df_st_ta_td_tm_tot,how='left')
df_train_sp_tot.shape

(6600, 48)

### **FINAL TRAINING SET**

In [137]:
df_train_sp_tot.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid', 'MostCommonMedication', 'MedicationNDCCount',
       'MostAllergyTypes', 'T2D_Diagnosis_Flag_Mode',
       'MostDiagnosisDescription', 'MostCommonICD9', 'MostStartYear',
       'MostDiagnosisGuid', 'MostPanelID', 'MostLabObservation',
       'MostHL7Identifier', 'MostAbnormalFlags', 'MostIsAbnormalValue',
       'MostStatus', 'MostLabTest', 'MostLabResultGuid', 'MostReportYear',
       'MostAncestorLabResultGuid', 'ConditionGuid',
       'MostPatientConditionGuid', 'MostCreatedYear', 'MostMedicationGuid',
       'MostPrescriptionGuid', 'MostPrescriptionYear', 'SmokingStatusGuid',
       'MostT2DSmokingFlag', 'MostDescription', 'MostNISTCode',
       'MostVisitYear', 'MostTranscriptGuid', 'AvgHeight', 'AvgWeight',
       'AvgBMI', 'AvgSystolicBP', 'AvgDiastolicBP', 'AvgRespiratoryRate',
       'AvgHeartRate', 'TranscriptGuid', 'MostTranscriptAllergyGuid',
       'MostTranscriptDiagnosisGuid', '

In [138]:
df_train_sp_tot.columns = df_train_sp_tot.columns.str.replace(r"^Most", "", regex=True)

In [139]:
df_train_sp_tot.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid', 'CommonMedication', 'MedicationNDCCount',
       'AllergyTypes', 'T2D_Diagnosis_Flag_Mode', 'DiagnosisDescription',
       'CommonICD9', 'StartYear', 'DiagnosisGuid', 'PanelID', 'LabObservation',
       'HL7Identifier', 'AbnormalFlags', 'IsAbnormalValue', 'Status',
       'LabTest', 'LabResultGuid', 'ReportYear', 'AncestorLabResultGuid',
       'ConditionGuid', 'PatientConditionGuid', 'CreatedYear',
       'MedicationGuid', 'PrescriptionGuid', 'PrescriptionYear',
       'SmokingStatusGuid', 'T2DSmokingFlag', 'Description', 'NISTCode',
       'VisitYear', 'TranscriptGuid', 'AvgHeight', 'AvgWeight', 'AvgBMI',
       'AvgSystolicBP', 'AvgDiastolicBP', 'AvgRespiratoryRate', 'AvgHeartRate',
       'TranscriptGuid', 'TranscriptAllergyGuid', 'TranscriptDiagnosisGuid',
       'TranscriptMedicationGuid', 'AllergyGuid'],
      dtype='object')

Checking shape.

In [140]:
df_train_sp_tot.shape

(6600, 48)

In [141]:
relevant_columns = [
    'PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth','State','PracticeGuid'
]

df_train_final_filtered = df_train_sp_tot[relevant_columns]

In [142]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import brier_score_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [143]:
label_encoder = LabelEncoder()
df_train_final_filtered['Gender'] = df_train_final_filtered['Gender'].astype(str)
df_train_final_filtered['State'] = df_train_final_filtered['State'].astype(str)
df_train_final_filtered['YearOfBirth'] = df_train_final_filtered['YearOfBirth'].astype(str)
df_train_final_filtered['PracticeGuid'] = df_train_final_filtered['PracticeGuid'].astype(str)

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2250894760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_final_filtered['Gender'] = df_train_final_filtered['Gender'].astype(str)
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2250894760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_final_filtered['State'] = df_train_final_filtered['State'].astype(str)
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2250894760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [144]:
df_train_final_filtered['Gender'] = label_encoder.fit_transform(df_train_final_filtered['Gender'])
df_train_final_filtered['State'] = label_encoder.fit_transform(df_train_final_filtered['State'])
df_train_final_filtered['YearOfBirth'] = label_encoder.fit_transform(df_train_final_filtered['YearOfBirth'])
df_train_final_filtered['PracticeGuid'] = label_encoder.fit_transform(df_train_final_filtered['PracticeGuid'])

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4194480410.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_final_filtered['Gender'] = label_encoder.fit_transform(df_train_final_filtered['Gender'])
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4194480410.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_final_filtered['State'] = label_encoder.fit_transform(df_train_final_filtered['State'])
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4194480410.py:3: SettingWithCopyWarning: 
A va

In [145]:
non_numeric_columns = df_train_final_filtered.select_dtypes(include=['object']).columns
for column in non_numeric_columns:
    print(f"Unique values in {column}: {df_train_final_filtered[column].unique()}")
    label_encoder = LabelEncoder()
    df_train_final_filtered[column] = label_encoder.fit_transform(df_train_final_filtered[column].astype(str))

Unique values in PatientGuid: ['00023761-9D8D-445B-874C-2424CC7CF620'
 '0005D9BD-0247-4F02-B7EE-7C1B44825FA1'
 '00110ABC-DAB9-49E3-A1C8-88BBF8D58109' ...
 'FFEF293B-6147-47F0-8F0B-76180C124205'
 'FFF2B406-9C3F-4A8B-8B27-7179B7D7FBC6'
 'FFF5C634-1CF7-40D5-BDF1-4FA2D62D9C63']


C:\Users\svkar\AppData\Local\Temp\ipykernel_824\198973311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_final_filtered[column] = label_encoder.fit_transform(df_train_final_filtered[column].astype(str))


In [146]:
df_train_final_filtered_imputed = df_train_final_filtered.fillna(0)

In [147]:
df_train_final_filtered_imputed.shape

(6600, 6)

In [148]:
from scipy.stats import zscore
def remove_outliers(df, z_threshold=3):
    for column in df.select_dtypes(include=['float64', 'int64']).columns:
        z_scores = pd.Series(zscore(df[column]), index=df.index)
        df = df[(z_scores.abs() < z_threshold) | z_scores.isna()]  # Retain NaNs as valid rows
    return df

In [149]:
df_train_cleaned = remove_outliers(df_train_final_filtered_imputed)

In [150]:
df_train_cleaned.shape

(6600, 6)

### **Test Prediction**

In [151]:
df_test_sp.columns

Index(['PatientGuid', 'Gender', 'YearOfBirth', 'State', 'PracticeGuid'], dtype='object')

In [152]:
df_test_sp_tot = df_test_sp.groupby("PatientGuid").agg( 
    Gender=("Gender", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    YearOfBirth=("YearOfBirth", lambda x: x.mode().iloc[0] if not x.mode().empty else None), 
    State=("State", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    PracticeGuid=("PracticeGuid", lambda x: x.mode().iloc[0] if not x.mode().empty else None),
).reset_index()

In [153]:
df_test_sp_tot.shape

(3348, 5)

In [154]:
df_train_cleaned.shape

(6600, 6)

In [155]:
X = df_train_cleaned.drop(columns=['PatientGuid', 'DMIndicator'])
y = df_train_cleaned['DMIndicator']

In [156]:
X.shape

(6600, 4)

In [157]:
y.shape

(6600,)

In [158]:
test_size = 3348 / 6601

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

In [160]:
df_test_sp_tot.shape

(3348, 5)

In [161]:
X_test.shape

(3348, 4)

In [162]:
X_train.shape

(3252, 4)

In [163]:
y_train.shape

(3252,)

In [164]:
X_test.shape

(3348, 4)

In [165]:
y_test.shape

(3348,)

In [166]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [167]:
print("Before scaling:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# After scaling:
print("After scaling:")
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)

Before scaling:
X_train shape: (3252, 4)
y_train shape: (3252,)
X_test shape: (3348, 4)
y_test shape: (3348,)
After scaling:
X_train_scaled shape: (3252, 4)
X_test_scaled shape: (3348, 4)


In [168]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_predict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss

In [169]:
base_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
base_model.fit(X_train_scaled, y_train)

GradientBoostingClassifier(random_state=42)

In [170]:
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'subsample': [0.7, 0.8, 1.0],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

gb_model = GradientBoostingClassifier(random_state=42)
random_search = RandomizedSearchCV(gb_model, param_grid, cv=5, scoring='neg_brier_score', n_iter=20, random_state=42, verbose=2)
random_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   2.0s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.7s
[CV] END learning_rate=0.01, max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   0.3s
[CV] END learning_r

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
                   n_iter=20,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2,
                                                          0.3],
                                        'max_depth': [3, 4, 5, 6, 7, 8],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 150, 200,
                                                         250],
                                        'subsample': [0.7, 0.8, 1.0]},
                   random_state=42, scoring='neg_brier_score', verbose=2)

In [171]:
best_gb_model = random_search.best_estimator_
calibrated_model = CalibratedClassifierCV(best_gb_model, method='isotonic', cv='prefit')
calibrated_model.fit(X_train_scaled, y_train)

CalibratedClassifierCV(cv='prefit',
                       estimator=GradientBoostingClassifier(learning_rate=0.05,
                                                            max_features='sqrt',
                                                            random_state=42,
                                                            subsample=0.7),
                       method='isotonic')

In [172]:
y_pred_proba = calibrated_model.predict_proba(X_test_scaled)[:, 1]
brier_score = brier_score_loss(y_test, y_pred_proba)
print(f"Final Calibrated Brier Score: {brier_score}")

Final Calibrated Brier Score: 0.14572124724946223


In [173]:
y_pred_proba.size

3348

In [174]:
df_train_cleaned.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid'],
      dtype='object')

In [175]:
df_test_sp_tot.columns

Index(['PatientGuid', 'Gender', 'YearOfBirth', 'State', 'PracticeGuid'], dtype='object')

In [176]:
results = pd.DataFrame({
    'PatientGuid': df_test_sp_tot['PatientGuid'],  
    'DMIndicatorForecast': y_pred_proba  
})

In [177]:
output_file_path = "C:/Users/svkar/Downloads/test_SyncPatientForecast.csv"
results.to_csv(output_file_path, index=False)
print(f"File saved successfully at {output_file_path}")

File saved successfully at C:/Users/svkar/Downloads/test_SyncPatientForecast.csv


### Saving File

In [178]:
df_test_sp_aligned = df_test_sp_tot.iloc[:X_test_scaled.shape[0]]

In [179]:
results = pd.DataFrame({
    'PatientGuid': df_test_sp_tot['PatientGuid'],  
    'DMIndicatorForecast': y_pred_proba  
})

In [180]:
results.to_csv('test_SyncPatientForecast.csv', index=False)

In [181]:
print(results)

                               PatientGuid  DMIndicatorForecast
0     000B4862-7CE7-4EC5-8043-A97FCD74BD78             0.004484
1     003CEE1F-5BF5-4171-9284-F5464EC12D41             0.004484
2     004382BD-E31F-4091-8DE5-E86A59D70C2D             0.004484
3     004EA6A7-6159-4D70-B06A-C0391D8623BD             0.081571
4     0063B34F-C2C4-423A-A144-E51F7149253A             0.333333
...                                    ...                  ...
3343  FF9BA1EB-03AB-47FE-B110-B559D8E0835B             0.023077
3344  FFA25126-98E0-46E2-9B80-FBEEEA1C7D71             0.812500
3345  FFB3BB66-F74A-462E-A9B8-F982857B4CB3             0.212871
3346  FFC8E19F-A199-46A3-895A-E3C07FBAC809             0.004484
3347  FFCD70A8-72FC-4F06-AC19-D9962B2CA95E             0.473684

[3348 rows x 2 columns]


In [182]:
import os
print(os.getcwd())

C:\Users\svkar\Data Science in Healthcare


In [183]:
from IPython.display import FileLink
results.to_csv('test_SyncPatientForecast.csv', index=False)
FileLink('test_SyncPatientForecast.csv')

C:\Users\svkar\Data Science in Healthcare\test_SyncPatientForecast.csv

In [184]:
output_file_path = "C:/Users/svkar/OneDrive/Documents"

In [185]:
output_file_path = "C:/Users/svkar/Downloads/test_SyncPatientForecast.csv"
results.to_csv(output_file_path, index=False)
print(f"File saved successfully at {output_file_path}")

File saved successfully at C:/Users/svkar/Downloads/test_SyncPatientForecast.csv


### Stacking Model

In [186]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [187]:
model = GradientBoostingClassifier()
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}

In [188]:
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

In [189]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train_poly, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.01, 0.1, 0.5],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]})

In [190]:
best_model = grid_search.best_estimator_
calibrated_model = CalibratedClassifierCV(best_model, method='sigmoid')
calibrated_model.fit(X_train_poly, y_train)

CalibratedClassifierCV(estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                            n_estimators=50))

In [191]:
y_pred_calibrated = calibrated_model.predict_proba(X_test_poly)[:, 1]

In [192]:
brier_score = brier_score_loss(y_test, y_pred_calibrated)
print(f"Final Calibrated Brier Score: {brier_score}")

Final Calibrated Brier Score: 0.14303585695691762


In [193]:
results = pd.DataFrame({
    'PatientGuid': df_test_sp_tot['PatientGuid'],  
    'DMIndicatorForecast': y_pred_calibrated  
})

In [194]:
output_file_path = "C:/Users/svkar/Downloads/test_SyncPatientForecast_2.csv"
results.to_csv(output_file_path, index=False)
print(f"File saved successfully at {output_file_path}")

File saved successfully at C:/Users/svkar/Downloads/test_SyncPatientForecast_2.csv


#### Model with Flags Included

In [195]:
df_train_sp_tot.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid', 'CommonMedication', 'MedicationNDCCount',
       'AllergyTypes', 'T2D_Diagnosis_Flag_Mode', 'DiagnosisDescription',
       'CommonICD9', 'StartYear', 'DiagnosisGuid', 'PanelID', 'LabObservation',
       'HL7Identifier', 'AbnormalFlags', 'IsAbnormalValue', 'Status',
       'LabTest', 'LabResultGuid', 'ReportYear', 'AncestorLabResultGuid',
       'ConditionGuid', 'PatientConditionGuid', 'CreatedYear',
       'MedicationGuid', 'PrescriptionGuid', 'PrescriptionYear',
       'SmokingStatusGuid', 'T2DSmokingFlag', 'Description', 'NISTCode',
       'VisitYear', 'TranscriptGuid', 'AvgHeight', 'AvgWeight', 'AvgBMI',
       'AvgSystolicBP', 'AvgDiastolicBP', 'AvgRespiratoryRate', 'AvgHeartRate',
       'TranscriptGuid', 'TranscriptAllergyGuid', 'TranscriptDiagnosisGuid',
       'TranscriptMedicationGuid', 'AllergyGuid'],
      dtype='object')

In [196]:
columns_to_keep = ['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State', 'PracticeGuid','T2D_Diagnosis_Flag_Mode','AbnormalFlags','T2DSmokingFlag']
df_train_sp_tot_filtered = df_train_sp_tot[columns_to_keep]

In [197]:
df_train_sp_tot_filtered.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid', 'T2D_Diagnosis_Flag_Mode', 'AbnormalFlags',
       'T2DSmokingFlag'],
      dtype='object')

In [198]:
flag_columns = ['T2D_Diagnosis_Flag_Mode', 'AbnormalFlags', 'T2DSmokingFlag']

In [199]:
df_train_sp_tot_filtered = df_train_sp_tot_filtered.copy()

df_train_sp_tot_filtered[flag_columns] = df_train_sp_tot_filtered[flag_columns].apply(pd.to_numeric, errors='coerce')

df_train_sp_tot_filtered[flag_columns] = df_train_sp_tot_filtered[flag_columns].fillna(0)

df_train_sp_tot_filtered['Flag_Sum'] = df_train_sp_tot_filtered[flag_columns].sum(axis=1)

df_train_sp_tot_filtered['DMIndicator_Weighted'] = (
    df_train_sp_tot_filtered['DMIndicator'] * (1 + df_train_sp_tot_filtered['Flag_Sum'])
)

In [200]:
df_train_sp_tot_filtered.shape

(6600, 11)

In [201]:
df_train_sp_tot_filtered.columns

Index(['PatientGuid', 'DMIndicator', 'Gender', 'YearOfBirth', 'State',
       'PracticeGuid', 'T2D_Diagnosis_Flag_Mode', 'AbnormalFlags',
       'T2DSmokingFlag', 'Flag_Sum', 'DMIndicator_Weighted'],
      dtype='object')

In [202]:
columns_to_keep = ['PatientGuid', 'DMIndicator_Weighted', 'Gender', 'YearOfBirth', 'State', 'PracticeGuid']
df_train_sp_tot_filtered_new = df_train_sp_tot_filtered[columns_to_keep]

In [203]:
label_encoder = LabelEncoder()
df_train_sp_tot_filtered_new['Gender'] = df_train_sp_tot_filtered_new['Gender'].astype(str)
df_train_sp_tot_filtered_new['State'] = df_train_sp_tot_filtered_new['State'].astype(str)
df_train_sp_tot_filtered_new['YearOfBirth'] = df_train_sp_tot_filtered_new['YearOfBirth'].astype(str)
df_train_sp_tot_filtered_new['PracticeGuid'] = df_train_sp_tot_filtered_new['PracticeGuid'].astype(str)

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4163860734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sp_tot_filtered_new['Gender'] = df_train_sp_tot_filtered_new['Gender'].astype(str)
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4163860734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sp_tot_filtered_new['State'] = df_train_sp_tot_filtered_new['State'].astype(str)
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\4163860734.py:4: SettingWithCopyWarning: 
A value is trying 

In [204]:
df_train_sp_tot_filtered_new['Gender'] = label_encoder.fit_transform(df_train_sp_tot_filtered_new['Gender'])
df_train_sp_tot_filtered_new['State'] = label_encoder.fit_transform(df_train_sp_tot_filtered_new['State'])
df_train_sp_tot_filtered_new['YearOfBirth'] = label_encoder.fit_transform(df_train_sp_tot_filtered_new['YearOfBirth'])
df_train_sp_tot_filtered_new['PracticeGuid'] = label_encoder.fit_transform(df_train_sp_tot_filtered_new['PracticeGuid'])

C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2173598551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sp_tot_filtered_new['Gender'] = label_encoder.fit_transform(df_train_sp_tot_filtered_new['Gender'])
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2173598551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_sp_tot_filtered_new['State'] = label_encoder.fit_transform(df_train_sp_tot_filtered_new['State'])
C:\Users\svkar\AppData\Local\Temp\ipykernel_824\2173598551.py:3: SettingWi

In [205]:
df_train_sp_tot_filtered_imputed = df_train_sp_tot_filtered_new.fillna(0)

In [206]:
df_train_cleaned = remove_outliers(df_train_sp_tot_filtered_imputed)

In [207]:
X = df_train_cleaned.drop(columns=['PatientGuid', 'DMIndicator_Weighted'])
y = df_train_cleaned['DMIndicator_Weighted']

In [208]:
test_size = 3348 / 6600

In [209]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

In [210]:
X_train.shape

(3251, 4)

In [211]:
y_test.shape

(3348,)

In [212]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [213]:
base_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
base_model.fit(X_train_scaled, y_train)

GradientBoostingClassifier(random_state=42)

In [214]:
param_grid = {
    'n_estimators': [50, 100, 150, 200, 250],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6, 7, 8],
    'subsample': [0.7, 0.8, 1.0],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

gb_model = GradientBoostingClassifier(random_state=42)
random_search = RandomizedSearchCV(gb_model, param_grid, cv=5, scoring='neg_brier_score', n_iter=20, random_state=42, verbose=2)
random_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   2.0s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.7s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.8s
[CV] END learning_rate=0.1, max_depth=8, max_features=None, min_samples_leaf=4, min_samples_split=5, n_estimators=200, subsample=0.8; total time=   1.9s
[CV] END learning_rate=0.01, max_depth=5, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100, subsample=1.0; total time=   0.4s
[CV] END learning_r

RandomizedSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
                   n_iter=20,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2,
                                                          0.3],
                                        'max_depth': [3, 4, 5, 6, 7, 8],
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 150, 200,
                                                         250],
                                        'subsample': [0.7, 0.8, 1.0]},
                   random_state=42, scoring='neg_brier_score', verbose=2)

In [215]:
best_model = grid_search.best_estimator_
calibrated_model = CalibratedClassifierCV(best_model, method='sigmoid')
calibrated_model.fit(X_train_scaled, y_train)

CalibratedClassifierCV(estimator=GradientBoostingClassifier(learning_rate=0.01,
                                                            n_estimators=50))

In [216]:
y_pred = calibrated_model.predict_proba(X_test_scaled)[:, 1]

In [217]:
brier_score = brier_score_loss(y_test, y_pred)
print(f"Final Calibrated Brier Score: {brier_score}")

Final Calibrated Brier Score: 0.14320045013080981


In [218]:
output_file_path = "C:/Users/svkar/Downloads/test_SyncPatientForecast_3.csv"
results.to_csv(output_file_path, index=False)
print(f"File saved successfully at {output_file_path}")

File saved successfully at C:/Users/svkar/Downloads/test_SyncPatientForecast_3.csv
